# Model 08

A triple u-net like architecture.

## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, MaxPooling3D, Conv3D, 
                          UpSampling3D, BatchNormalization, Concatenate)
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import util
importlib.reload(util)
import preprocessing
importlib.reload(preprocessing)
import datagen
importlib.reload(datagen)
import modelutil
importlib.reload(modelutil)

SEED = 0
EPOCHS = 100
BATCH_SIZE = 4
PATCH_SHAPE = (32, 32, 32)

MODEL_NAME = 'model_08'

DATA_DIR = Path('/data2').expanduser()
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'

MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_LOG_DIR = PROJECT_DATA_DIR / 'tensorboard' / MODEL_NAME
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_LOG_DIR, TMP_DIR, PP_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()

%load_ext autoreload
%autoreload 2

## Data Generation

In [ ]:
train_gen, val_gen = datagen.get_nifti_datagens(preprocessed_metadata_path=PP_MD_PATH,
                                                batch_size=BATCH_SIZE, crop_shape=PATCH_SHAPE)


## Build Model


In [ ]:
def build_residual_encoder_decoder_block(x, n_a, n_d=1):

    x = BatchNormalization()(x)
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x) # (32, 32, 32, 16)

    if n_d > 0:
        x_e = x # shape: (32, 32, 32, 16)
        x_e = MaxPooling3D(padding='same')(x_e) # shape: (16, 16, 16, 16)
        x_e = build_residual_encoder_decoder_block(x_e, n_a, n_d - 1) # recursive call
        x_d = UpSampling3D()(x_e) # shape (32, 32, 32, 16)
        x = Concatenate()([x, x_d]) # residual join.  shape (32, 32, 32, 32)
        x = BatchNormalization()(x)
        x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x) # (32, 32, 32, 16)
    
    return x
def build_model(input_shape, n_a=16, n_r=2, n_d=2):
    '''
    3D convolutional autoencoder that treats u-net architecture as a residual block.
    
    1 poolings reduce input from shape to shape/2, which in 3d is 1/8th the size of the original shape,
    a very respectable compression factor.
    '''

    x_input = Input(shape=input_shape) # shape:  (32, 32, 32, 1) == 32768
    x = x_input
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x) # shape: (32, 32, 32, 16)
    
    for i in range(n_r):
        x = build_residual_encoder_decoder_block(x, n_a=n_a, n_d=n_d)
    
    y = Conv3D(1, kernel_size=(3, 3, 3), padding='same', activation='sigmoid')(x) # shape: (32, 32, 32, 1) = 32768
    
    model = Model(inputs=x_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
   
   

In [ ]:

model = build_model(PATCH_SHAPE + (1,), n_a=32, n_r=4, n_d=4)
modelutil.display(model)

## Train and Evaluate Model

- Add callbacks to save model every 20 epochs and to log performance stats every epoch, so we have the results saved somewhere for charting.


In [ ]:
history, log_path = modelutil.train_model(
    model, train_gen, val_gen, epochs=400, batch_size=BATCH_SIZE, models_dir=MODELS_DIR, model_name=MODEL_NAME, 
    log_dir=LOG_DIR, tensorboard_log_dir=TENSORBOARD_LOG_DIR)
# history, log_path = train_model_epoch(train_gen, val_gen, epoch=100, epochs=200)

## Visualize Training Progress

In [ ]:
# read metrics from the log file
# log_path = LOG_DIR / (model_name + '_2018-04-26T17:29:02.902740_log.csv')

metrics = pd.read_csv(log_path)

In [ ]:
print(pd.concat([metrics[::10], metrics[-1:]])) # every 10th metric and the last one

In [ ]:
# Plot Training and Validation Accuracy 
axes = plt.gca()
axes.set_ylim([0.0,1.0]) # Show results on 0..1 range
plt.plot(metrics["acc"])
plt.plot(metrics["val_acc"])
plt.legend(['Training Accuracy', "Validation Accuracy"])
plt.show()

# Plot Training and Validation Loss
plt.plot(metrics["loss"])
plt.plot(metrics["val_loss"])
plt.legend(['Training Loss', "Validation Loss"])
plt.show()



## Show Effect of Autoencoder Training

Use models from different training epochs to encode images.


In [ ]:
for viz in modelutil.vizualize_predictions_by_epochs(MODELS_DIR, MODEL_NAME, [1, 10, 200], train_gen, val_gen, step=1):
    display(viz)
    
